In [26]:
import numpy as np
import pandas as pd
import pickle

In [27]:
# Import my modules.
import sys, os
from pathlib import Path
current_dir = os.path.join(Path().resolve())
sys.path.append(str(current_dir) + '/../')

from modules import utils
from modules import models
from modules import preprocess

import importlib
for m in [utils, models, preprocess]:
    importlib.reload(m)

In [28]:
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 150)

In [29]:
mbd = 'microbusiness_density'

df_train, df_test, df_subm = utils.load_dataset(subm='../submission/submission_13730_trend.csv')
df_all, df_census = utils.merge_dataset(df_train, df_test, unemploy=False, census=True, co_est=False, coord=True, add_location=True, 
                                        fix_pop=True, merge41=True, df_subm=df_subm)

merge scale=41 of df_submission to df_all.


In [33]:
params={
        "act_thre": 140,
        "abs_thre": 0,
        "USE_LAG": 8,
        "USE_TREND": True,
        "USE_SHORT_TREND": True,
        "USE_SEASON": True,
        "blacklist": [],
        "blacklistcfips": [],
        "clip": (-0.0044, 0.0046),
        "model": 'lgb',
        "max_window": 12,
        "start_max_scale": 41,
        "start_all_dict": 40,
        "smooth_method": 'v3',
        "save_output_dic": True,
        "light": True,
    }
trend_params = {
        "high_trend_params": {
            1: {
                'params':{
                    'n':12,
                    'thre':10,
                    'thre_r':0,
                    'lower_bound': 10000,
                    'upper_bound': 99999999,
                    'use_regularize': True,
                    'v_regularize': [0.001, 0],
                    'trend_clip': [0.95, 1.05],
                    'v_clip':[0.98, 1.02],

                },
                'method': 'mean'
            },
            2: {
                'params':{
                    'n':13,
                    'thre':11,
                    'thre_r':0.001,
                    'lower_bound': 10000,
                    'upper_bound': 99999999,
                    'use_regularize': True,
                    'v_regularize': [0.002, 0.001],
                    'trend_clip': [0.95, 1.05],
                    'v_clip':[0.99, 1.01]
                },
                'method': 'mean'
            },
            3: {
                'params':{
                    'n':14,
                    'thre':12,
                    'thre_r':0.002,
                    'lower_bound': 10000,
                    'upper_bound': 99999999,
                    'use_regularize': True,
                    'v_regularize': [0.004, 0.003],
                    'trend_clip': [0.95, 1.05],
                    'v_clip':[0.995, 1.005]
                },
                'method': 'mean'
            },
            4: {
                'params':{
                    'n':15,
                    'thre':13,
                    'thre_r':0.003,
                    'lower_bound': 10000,
                    'upper_bound': 99999999,
                    'use_regularize': True,
                    'v_regularize': [0.008, 0.005],
                    'trend_clip': [0.95, 1.05],
                    'v_clip':[0.995, 1.005]
                },
                'method': 'mean'
            }
        },
        "low_trend_params": {
            1: {
                'params':{
                    'n':8,
                    'thre':7,
                    'thre_r':0,
                    'lower_bound': 2000,
                    'upper_bound': 10000,
                    'use_regularize': True,
                    'v_regularize': [0.001, 0],
                    'trend_clip': [0.95, 1.05],
                    'v_clip':[0.98, 1.02]
                },
                'method': 'mean'
            },
            2: {
                'params':{
                    'n':9,
                    'thre':8,
                    'thre_r':0.001,
                    'lower_bound': 2000,
                    'upper_bound': 10000,
                    'use_regularize': True,
                    'v_regularize': [0.002, 0.001],
                    'trend_clip': [0.95, 1.05],
                    'v_clip':[0.99, 1.01]
                },
                'method': 'mean'
            },
            3: {
                'params':{
                    'n':10,
                    'thre':9,
                    'thre_r':0.002,
                    'lower_bound': 2000,
                    'upper_bound': 10000,
                    'use_regularize': True,
                    'v_regularize': [0.004, 0.003],
                    'trend_clip': [0.95, 1.05],
                    'v_clip':[0.995, 1.005]
                },
                'method': 'mean'
            },
            4: {
                'params':{
                    'n':11,
                    'thre':10,
                    'thre_r':0.003,
                    'lower_bound': 2000,
                    'upper_bound': 10000,
                    'use_regularize': True,
                    'v_regularize': [0.008, 0.005],
                    'trend_clip': [0.95, 1.05],
                    'v_clip':[0.995, 1.005]
                },
                'method': 'mean'
            }
        },
        "short_trend_params": {
            1: {
                'params':{
                    'n':4,
                    'thre':4,
                    'thre_r':0.003,
                    'lower_bound': 2000,
                    'upper_bound': 99999999,
                    'use_regularize': True,
                    'v_regularize': [0.002, 0.001],
                    'trend_clip': [0.95, 1.05],
                    'v_clip':[0.98, 1.02]
                },
                'method': 'mix'
            },
            2: {
                'params':{
                    'n':5,
                    'thre':5,
                    'thre_r':0.005,
                    'lower_bound': 2000,
                    'upper_bound': 99999999,
                    'use_regularize': True,
                    'v_regularize': [0.002, 0.001],
                    'trend_clip': [0.95, 1.05],
                    'v_clip':[0.99, 1.01]
                },
                'method': ''
            },
            3: {
                'params':{
                    'n':6,
                    'thre':6,
                    'thre_r':0.006,
                    'lower_bound': 2000,
                    'upper_bound': 99999999,
                    'use_regularize': True,
                    'v_regularize': [0.01, 0.01],
                    'trend_clip': [0.95, 1.05],
                    'v_clip':[0.995, 1.005]
                },
                'method': ''
            },
            4: {
                'params':{
                    'n':7,
                    'thre':7,
                    'thre_r':0.007,
                    'lower_bound': 2000,
                    'upper_bound': 99999999,
                    'use_regularize': True,
                    'v_regularize': [0.02, 0.02],
                    'trend_clip': [0.95, 1.05],
                    'v_clip':[0.995, 1.005]
                },
                'method': ''
            }
        }
}
season_params = {
    "abs_thre": [-0.006, 0.006],
    "active_thre": 2000,
    "v_clip": [-0.01, 0.01],
    "method": "trend_mean"
}

## バリデーション

### 前の予測結果を用いて予測していく

In [24]:
accum_pred = models.LgbmBaseline('params_accum',df_subm, df_all, df_census, params=params, trend_params=trend_params, season_params=season_params)
accum_pred.accum_validation()

create df_all_dict[32] and df_all_dict_original[32]
add lag features: max_scale=32
smooth_outlier: max_scale=32
used method: v3
# of fixed cfips: 1123
# of fixed value: 2015
create df_all_dict[33] and df_all_dict_original[33]
add lag features: max_scale=33
smooth_outlier: max_scale=33
used method: v3
# of fixed cfips: 1132
# of fixed value: 2041
create df_all_dict[34] and df_all_dict_original[34]
add lag features: max_scale=34
smooth_outlier: max_scale=34
used method: v3
# of fixed cfips: 1140
# of fixed value: 2081
create df_all_dict[35] and df_all_dict_original[35]
add lag features: max_scale=35
smooth_outlier: max_scale=35
used method: v3
# of fixed cfips: 1152
# of fixed value: 2114
create df_all_dict[36] and df_all_dict_original[36]
add lag features: max_scale=36
smooth_outlier: max_scale=36
used method: v3
# of fixed cfips: 1152
# of fixed value: 2128
create df_all_dict[37] and df_all_dict_original[37]
add lag features: max_scale=37
smooth_outlier: max_scale=37
used method: v3
# 

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:7246: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result[mask] = np.nan


# of cfips that have high trend : 23
use method:  mean
# of cfips that have low trend : 15
use method:  replace


AttributeError: 'NoneType' object has no attribute 'get'

## Submissionの作成

### 予測結果を活用して予測

In [ ]:
test_subm = models.LgbmBaseline('test_subm_accum', df_subm, df_all, df_census, params=params, trend_params=trend_params, season_params=season_params)
test_subm.create_submission(target_scale=[42, 43,44,45], save=False)

create df_all_dict[40] and df_all_dict_original[40]
add lag features: max_scale=40
smooth_outlier: max_scale=40
used method: v3
# of fixed cfips: 1189
# of fixed value: 2310
create df_all_dict[41] and df_all_dict_original[41]
add lag features: max_scale=41
smooth_outlier: max_scale=41


In [9]:
df_submission =test_subm.df_submission
df_merged = utils.compare_submission(df_subm, 'submission_13730_trend')
df_merged['smape'].sum()

0.0

In [8]:
# ensemble
0.01261112205424169

# trend_multi有り
0.03200090765098272

0.03200090765098272

In [24]:
id1 = (df_all['county']==20)|(df_all['county']==40)
id2 = (df_all['scale']==41)&id1

In [26]:
sum(id2)

2